In [15]:
import pandas
import os
import json
import pprint
from feature.feature_concat import feature_extraction
from feature.setup import pipeline_setup
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv("test.env", override=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + os.environ.get("SERVICE_ACCOUNT_PATH")

In [ ]:
# test 1, 4-5 minutes
id = "elJuUGruxOU"
keyword = "test"
pipeline_setup(id)
df_test = feature_extraction(id, keyword)
pprint.pprint(df_test)
with open("temp/feature/" + id + ".json", "w") as outfile:
    json.dump(df_test, outfile)

In [ ]:
# test 2, 7 mins
id = "nMLRpJDXR-4"
keyword = "test"
pipeline_setup(id)
df_test = feature_extraction(id, keyword)
pprint.pprint(df_test)
with open("temp/feature/" + id + ".json", "w") as outfile:
    json.dump(df_test, outfile)